In [ ]:
#pip install typer==0.3.0

In [1]:
import fitz 
import re
import json 
from collections import defaultdict
import spacy
from nltk.corpus import stopwords, words
import string 
from tqdm import tqdm 

In [2]:
doc = fitz.open('data/original_data/gdpr_text.pdf')
print('Number of Pages available:', doc.page_count)
full_text = ' '.join(page.get_text() for page in doc)
print('Number of Tokens:', len(full_text))

Number of Pages available: 88
Number of Tokens: 360940


In [3]:
all_chapters = re.findall(r'CHAPTER .+\n', full_text)
all_articles = re.findall(r'\nArticle [0-9]+ \n', full_text)
gdpr_chunked = defaultdict(dict)

chapter_i_pos = re.search(fr'{all_chapters[0]}', full_text).span()
gdpr_chunked['Legislative acts']['Regulation']=full_text[36:chapter_i_pos[0]].replace('\n', '').strip()
previous_start = chapter_i_pos[1]

for ch in range(len(all_chapters)):
    if ch != len(all_chapters)-1:
        end_part = re.search(fr'{all_chapters[ch+1]}', full_text).span()[0]
        next_start = re.search(fr'{all_chapters[ch+1]}', full_text).span()[1]
        relevant_text = full_text[previous_start:end_part]
    else:
        relevant_text = full_text[previous_start:]
        
    chapt_name_idx = relevant_text.find('\n')
    for art in range(len(all_articles)-1):
        if re.search(fr'{all_articles[art]}', relevant_text)!=None:
            start_part_art = re.search(fr'{all_articles[art]}', relevant_text).span()[1]
            if art != len(all_chapters)-1 and re.search(fr'{all_articles[art+1]}', relevant_text)!=None:
                end_part_art = re.search(fr'{all_articles[art+1]}', relevant_text).span()[0]
                art_text = relevant_text[start_part_art:end_part_art]
                art_name_idx = art_text.find('\n')
                gdpr_chunked[relevant_text[:chapt_name_idx].strip()][art_text[:art_name_idx].strip()]=art_text[art_name_idx:].replace('\n', '').strip()
            else: 
                art_text=relevant_text[start_part_art:]
                art_name_idx = art_text.find('\n')
                gdpr_chunked[relevant_text[:chapt_name_idx].strip()][art_text[:art_name_idx].strip()]=art_text[art_name_idx:].replace('\n', '').strip()
    
    previous_start = next_start

In [4]:
with open('data/chunked_data/gdpr_chunk.json', 'w') as f: 
    json.dump(gdpr_chunked, f)

In [5]:
nlp = spacy.load("en_core_web_sm", disable=['ner', 'textcat'])
clean_stopwords = set(stopwords.words('english')).difference(set('not'))
clean_punctuation = ''.join(set(string.punctuation).difference(set(['()', '(', ')'])))

In [6]:
def lemmatize(text):
    return ' '.join(token.lemma_ for token in nlp(text))

In [7]:
gdpr_clean = defaultdict(dict)
for k in tqdm(gdpr_chunked.keys()):
    for sub_k in gdpr_chunked[k].keys():
        text = gdpr_chunked[k][sub_k]
        #Remove punctuation
        text = text.translate(str.maketrans('', '', '.,;:[]()\'')) 
        #Remove extra whitespaces, stopwords and lemmatize
        text = ' '.join(i.lower().strip() for i in lemmatize(text).split() if i not in clean_stopwords and len(i)>=2)
        gdpr_clean[k][sub_k] = text

100%|██████████| 12/12 [00:04<00:00,  2.95it/s]


In [8]:
with open('data/chunked_data/gdpr_clean.json', 'w') as f: 
    json.dump(gdpr_clean, f)